# Notebook: Train Model

## Packages

In [1]:
from load_dataset_folds import load_dataset_folds
from ACD import train_ACD_model
from OTE import train_OTE_model
import pandas as pd
import numpy as np
import constants
import warnings
import shutil
import torch
import json
import sys

* Todo: "output2" umbenennen ✅
* Todo: Evaluationsskript anpassen ✅
* Todo: "OTD" entfernen ✅
* Todo: Metrics speichern ✅
* Todo: Pfade ändern real/fake data ✅
* Todo: Nochmal prüfen, ob korrekte daten geladen werden ✅
* Todo: Schauen, wieso der erste Term mehrere Tags erhält ✅
* Todo: F1 macro brechnen ✅
* Metriken bei acd checken und schauen, ob evtl standardmethoden verwendet werden können ✅
* Todo: Prüfen, ob es Text gibt, der sowohl in Train als auch Test ist ✅

## Parameters

In [2]:
LLM_NAME = "Llama13B"
N_REAL = 500
N_SYNTH = 0
TARGET = "aspect_term" # "aspect_term", "aspect_category"
LLM_SAMPLING = "fixed"

## Settings

In [3]:
# Set seeds
torch.manual_seed(constants.RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(constants.RANDOM_SEED)

# Ignore warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="transformers.optimization")
torch.device("mps")

# Disable Pycache
sys.dont_write_bytecode = True

## Code

In [4]:
train_dataset, test_dataset = load_dataset_folds(LLM_NAME, N_REAL, N_SYNTH, LLM_SAMPLING)

### Load Model

In [5]:
if TARGET == "aspect_category":
   results = train_ACD_model(LLM_NAME, N_REAL, N_SYNTH, TARGET, LLM_SAMPLING, train_dataset, test_dataset)

In [10]:
if TARGET == "aspect_term":
    results = train_OTE_model(LLM_NAME, N_REAL, N_SYNTH, TARGET, LLM_SAMPLING, train_dataset, test_dataset)

example: {'text': 'Suppe war nicht heiß, kümmerte den Service aber nicht.', 'aspect_category': 'FOOD', 'tags': [{'end': 5, 'start': 0, 'tag_with_polarity': 'FOOD-POSITIVE', 'tag_with_polarity_and_type': 'FOOD-POSITIVE-explicit', 'text': 'Suppe', 'type': 'label-explicit', 'label': 'FOOD', 'polarity': 'POSITIVE'}], 'id': '9372f1ee-58b3-425e-8a6b-775627bbd424'}
one-hot: [[1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1

Exception: This is a custom error message.

### Save Results

In [ ]:
with open(f'results_json/results_{LLM_NAME}_real{N_REAL}_synth{N_SYNTH}_{TARGET}_{LLM_SAMPLING}.json', 'w') as json_file:
    json.dump(results, json_file)

df = pd.DataFrame([results])
df.to_csv(f'results_csv/results_{LLM_NAME}_real{N_REAL}_synth{N_SYNTH}_{TARGET}_{LLM_SAMPLING}.csv', index=False)

In [ ]:
results

{'LLM_NAME': 'Llama13B',
 'N_REAL': 500,
 'N_SYNTH': 0,
 'TARGET': 'aspect_category',
 'LLM_SAMPLING': 'fixed',
 'eval_loss': 0.5155795216560364,
 'eval_accuracy': 0.2,
 'eval_hamming_loss': 0.1852,
 'eval_f1_macro': 0.18387096774193548,
 'eval_f1_micro': 0.42484472049689437,
 'eval_f1_weighted': 0.29146738577388825,
 'eval_precision_GENERAL-IMPRESSION': 0.0,
 'eval_recall_GENERAL-IMPRESSION': 0.0,
 'eval_f1_GENERAL-IMPRESSION': 0.0,
 'eval_precision_FOOD': 0.0,
 'eval_recall_FOOD': 0.0,
 'eval_f1_FOOD': 0.0,
 'eval_precision_SERVICE': 1.0,
 'eval_recall_SERVICE': 0.8507462686567164,
 'eval_f1_SERVICE': 0.9193548387096774,
 'eval_precision_AMBIENCE': 0.0,
 'eval_recall_AMBIENCE': 0.0,
 'eval_f1_AMBIENCE': 0.0,
 'eval_precision_PRICE': 0.0,
 'eval_recall_PRICE': 0.0,
 'eval_f1_PRICE': 0.0,
 'runtime': 21.374308824539185,
 'runtime_formatted': '21s'}

### Remove useless folders

In [ ]:
try:
    shutil.rmtree("outputs")
except:
    pass